In [1]:
import os, sys
sys.path.insert(0, os.path.dirname(os.getcwd()))
sys.path.insert(0, os.path.dirname(os.path.dirname(os.getcwd())))
sys.path.insert(0, os.path.join(os.path.dirname(os.getcwd()), 'DeepUnitMatch'))
import UnitMatchPy.default_params as default_params
import UnitMatchPy.utils as util
import importlib
import matplotlib.pyplot as plt
import numpy as np
from DeepUnitMatch.utils import param_fun
from DeepUnitMatch.testing import test

In [2]:
# Getting the data the same way as UnitMatch

# Get default parameters, can add your own before or after!
param = default_params.get_default_param()

# Give the paths to the KS directories for each session
# If you don't have a dir with channel_positions.npy etc look at the detailed example for supplying paths separately 
KS_dirs = [r'\\znas\Subjects\AL032\2019-11-21\ephys_K1\pyKS\output', r'\\znas\Subjects\AL032\2019-11-22\ephys_K1\pyKS\output']

param['KS_dirs'] = KS_dirs
wave_paths, unit_label_paths, channel_pos = util.paths_from_KS(KS_dirs)
param = util.get_probe_geometry(channel_pos[0], param)

# STEP 0 from the UMPy example notebook
waveform, session_id, session_switch, within_session, good_units, param = util.load_good_waveforms(wave_paths, unit_label_paths, param, good_units_only = True) 

Using BombCell: cluster_bc_unitType
Using BombCell: cluster_bc_unitType


In [3]:
# Preprocess the DeepUnitMatch way and save as HDF5 files for each session in 'processed_waveforms'.
snippets, positions = param_fun.get_snippets(waveform, channel_pos, session_id)

In [4]:
# Load the neural net
model = test.load_trained_model(device="cpu")

# We'll store the preprocessed data here
data_dir = os.path.join(os.path.dirname(os.getcwd()), 'DeepUnitMatch', 'processed_waveforms')

# Pass the preprocessed data through the neural net
sim_matrix = test.inference(model, data_dir)

c:\Users\EnnyB\Documents\GitHub\DeepUnitMatch\UnitMatchPy\DeepUnitMatch\testing\test.py:25: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(read_path)


Initialised with 352 files in the dataset.
No. of experiment batches: 2


100%|██████████| 2/2 [00:02<00:00,  1.05s/it]
